# 2. Preprocessing and Feature Engineering

### Import libraries

In [137]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

### Find some important features

In [138]:
# Read cleaned csv file.
df         = pd.read_csv('../datasets/train_cleaned.csv', na_filter = False)
df_to_pred = pd.read_csv('../datasets/test_cleaned.csv', na_filter = False)

In [139]:
# Drop unnamed column.
df.drop('Unnamed: 0', axis = 1, inplace = True)
df_to_pred.drop('Unnamed: 0', axis = 1, inplace = True)

In [140]:
# See the correlation.
df.corr()[['saleprice']].sort_values(by='saleprice', ascending=False)

,saleprice
saleprice,1.000000
overall_qual,0.800207
gr_liv_area,0.697038
garage_area,0.649897
garage_cars,0.647781
total_bsmt_sf,0.629303
1st_flr_sf,0.618486
year_built,0.571849
year_remod/add,0.550370
full_bath,0.537969


In [141]:
# Drop 'ID', 'PID' columns. It is meaningless to predict saleprice.
df.drop(columns=['id', 'pid'], inplace=True)
df_to_pred.drop(columns=['id', 'pid'], inplace=True)

- Usually, ordinal values represents the order or quality and those are related to saleprice.

### This is the list of the column name of ordinal data from the data description.

In [142]:
# List of columns which have ordinal values.
# We will numerized them and get dummies.
ordinal_list = ['lot_shape',
               'utilities',
               'land_slope',
               'overall_qual',
               'overall_cond',
               'exter_qual',
               'exter_cond',
               'bsmt_qual',
               'bsmt_cond',
               'bsmt_exposure',
               'bsmtfin_type_1',
               'bsmtfin_type_2',
               'heating_qc',
               'electrical',
               'kitchen_qual',
               'functional',
               'fireplace_qu',
               'garage_finish',
               'garage_qual',
               'garage_cond',
               'paved_drive',
               'fence']

In [143]:
# List of columns which have nominal values.
nominal_list = ['ms_subclass',
               'ms_zoning',
               'street',
               'land_contour',
               'lot_config',
               'neighborhood',
               'condition_1',
               'condition_2',
               'bldg_type',
               'house_style',
               'roof_style', 
               'roof_matl',
               'exterior_1st', 
               'exterior_2nd',
               'mas_vnr_type',
               'foundation',
               'heating',
               'central_air',
               'garage_type',
               'sale_type']

### Check each column and numerize ordinal values

#### lot shape column

In [144]:
df.lot_shape.value_counts()

Reg    1295
IR1     692
IR2      55
IR3       9
Name: lot_shape, dtype: int64

In [145]:
df['lot_shape'] = df['lot_shape'].map({'Reg':3, 'IR1':2, 'IR2':1, 'IR3':0 })
df_to_pred['lot_shape'] = df_to_pred['lot_shape'].map({'Reg':3, 'IR1':2, 'IR2':1, 'IR3':0 })

#### utility column

In [146]:
df.utilities.value_counts()

AllPub    2049
NoSewr       1
NoSeWa       1
Name: utilities, dtype: int64

- Distribution is skewed too much. This feature won't affect the result much. 
- We will drop this column.

#### land slope column

In [147]:
df.land_slope.value_counts()

Gtl    1953
Mod      88
Sev      10
Name: land_slope, dtype: int64

In [148]:
df['land_slope'] = df['land_slope'].map({'Gtl':2, 'Mod':1, 'Sev':0})
df_to_pred['land_slope'] = df_to_pred['land_slope'].map({'Gtl':2, 'Mod':1, 'Sev':0})

#### exter qual, exter cond columns

In [149]:
df.exter_qual.value_counts() 
df.exter_cond.value_counts()

TA    1778
Gd     215
Fa      49
Ex       7
Po       2
Name: exter_cond, dtype: int64

In [150]:
df['exter_qual'] = df['exter_qual'].map({'Ex':4, 'Gd':3, 'TA':2, 'Fa':1, 'Po':0})
df['exter_cond'] = df['exter_cond'].map({'Ex':4, 'Gd':3, 'TA':2, 'Fa':1, 'Po':0})
df_to_pred['exter_qual'] = df_to_pred['exter_qual'].map({'Ex':4, 'Gd':3, 'TA':2, 'Fa':1, 'Po':0})
df_to_pred['exter_cond'] = df_to_pred['exter_cond'].map({'Ex':4, 'Gd':3, 'TA':2, 'Fa':1, 'Po':0})

#### bsmt qual, bsmt cond columns

In [151]:
df.bsmt_qual.value_counts()
df.bsmt_cond.value_counts()

TA    1834
Gd      89
Fa      65
NA      55
Po       5
Ex       3
Name: bsmt_cond, dtype: int64

In [152]:
df['bsmt_qual'] = df['bsmt_qual'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0})
df['bsmt_cond'] = df['bsmt_cond'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0})
df_to_pred['bsmt_qual'] = df_to_pred['bsmt_qual'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0})
df_to_pred['bsmt_cond'] = df_to_pred['bsmt_cond'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0})

#### bsmt exposure column

In [153]:
df.bsmt_exposure.value_counts()

No    1339
Av     288
Gd     203
Mn     163
NA      58
Name: bsmt_exposure, dtype: int64

In [154]:
df['bsmt_exposure'] = df['bsmt_exposure'].map({'Gd':4, 'Av':3, 'Mn':2, 'No':1, 'NA':0})
df_to_pred['bsmt_exposure'] = df_to_pred['bsmt_exposure'].map({'Gd':4, 'Av':3, 'Mn':2, 'No':1, 'NA':0})

#### bsmt fin type 1, type 2 columns

In [155]:
df.bsmtfin_type_1.value_counts()

GLQ    615
Unf    603
ALQ    293
BLQ    200
Rec    183
LwQ    102
NA      55
Name: bsmtfin_type_1, dtype: int64

In [156]:
df['bsmtfin_type_1'] = df['bsmtfin_type_1'].map({'GLQ':6, 'ALQ':5, 'BLQ':4, 'Rec':3, 'LwQ':2, 'Unf':1, 'NA':0})
df['bsmtfin_type_2'] = df['bsmtfin_type_2'].map({'GLQ':6, 'ALQ':5, 'BLQ':4, 'Rec':3, 'LwQ':2, 'Unf':1, 'NA':0})
df_to_pred['bsmtfin_type_1'] = df_to_pred['bsmtfin_type_1'].map({'GLQ':6, 'ALQ':5, 'BLQ':4, 'Rec':3, 'LwQ':2, 'Unf':1, 'NA':0})
df_to_pred['bsmtfin_type_2'] = df_to_pred['bsmtfin_type_2'].map({'GLQ':6, 'ALQ':5, 'BLQ':4, 'Rec':3, 'LwQ':2, 'Unf':1, 'NA':0})

#### heating qc columns

In [157]:
df.heating_qc.value_counts()

Ex    1065
TA     597
Gd     319
Fa      67
Po       3
Name: heating_qc, dtype: int64

In [158]:
df['heating_qc'] = df['heating_qc'].map({'Ex':4, 'Gd':3, 'TA':2, 'Fa':1, 'Po':0})
df_to_pred['heating_qc'] = df_to_pred['heating_qc'].map({'Ex':4, 'Gd':3, 'TA':2, 'Fa':1, 'Po':0})

#### electrical column

In [159]:
df.electrical.value_counts()

SBrkr    1868
FuseA     140
FuseF      35
FuseP       7
Mix         1
Name: electrical, dtype: int64

In [160]:
df['electrical'] = df['electrical'].map({'SBrkr':4, 'FuseA':3, 'FuseF':2, 'FuseP':1, 'Mix':1})
df_to_pred['electrical'] = df_to_pred['electrical'].map({'SBrkr':4, 'FuseA':3, 'FuseF':2, 'FuseP':1, 'Mix':1})

#### kitchen qual column

In [161]:
df.kitchen_qual.value_counts()

TA    1047
Gd     806
Ex     151
Fa      47
Name: kitchen_qual, dtype: int64

In [162]:
df['kitchen_qual'] = df['kitchen_qual'].map({'Ex':4, 'Gd':3, 'TA':2, 'Fa':1, 'Po':0})
df_to_pred['kitchen_qual'] = df_to_pred['kitchen_qual'].map({'Ex':4, 'Gd':3, 'TA':2, 'Fa':1, 'Po':0})

#### functional column

In [163]:
df.functional.value_counts()

Typ     1915
Min1      42
Min2      42
Mod       29
Maj1      12
Maj2       7
Sev        2
Sal        2
Name: functional, dtype: int64

In [164]:
df['functional'] = df['functional'].map({'Typ':7, 'Min1':6, 'Min2':5, 'Mod':4, 'Maj1':3, 'Maj2':2, 'Sal':1, 'Sev':0})
df_to_pred['functional'] = df_to_pred['functional'].map({'Typ':7, 'Min1':6, 'Min2':5, 'Mod':4, 'Maj1':3, 'Maj2':2, 'Sal':1, 'Sev':0})


#### fireplace qu column

In [165]:
df.fireplace_qu.value_counts()

NA    1000
Gd     523
TA     407
Fa      59
Ex      31
Po      31
Name: fireplace_qu, dtype: int64

In [166]:
df['fireplace_qu'] = df['fireplace_qu'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0})
df_to_pred['fireplace_qu'] = df_to_pred['fireplace_qu'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0})

#### garage finish column

In [167]:
df.garage_finish.value_counts()

Unf    849
RFn    579
Fin    509
NA     114
Name: garage_finish, dtype: int64

In [168]:
df['garage_finish'] = df['garage_finish'].map({'Fin':3, 'RFn':2, 'Unf':1, 'NA':0})
df_to_pred['garage_finish'] = df_to_pred['garage_finish'].map({'Fin':3, 'RFn':2, 'Unf':1, 'NA':0})

#### Garage qual, garage cond columns

In [169]:
df.garage_qual.value_counts()
df.garage_cond.value_counts()

TA    1868
NA     114
Fa      47
Gd      12
Po       8
Ex       2
Name: garage_cond, dtype: int64

In [170]:
df['garage_qual'] = df['garage_qual'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0})
df['garage_cond'] = df['garage_cond'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0})
df_to_pred['garage_qual'] = df_to_pred['garage_qual'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0})
df_to_pred['garage_cond'] = df_to_pred['garage_cond'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0})

#### paved drive column

In [171]:
df.paved_drive.value_counts()

Y    1861
N     151
P      39
Name: paved_drive, dtype: int64

In [172]:
df['paved_drive'] = df['paved_drive'].map({'Y':2, 'P':1, 'N':0})
df_to_pred['paved_drive'] = df_to_pred['paved_drive'].map({'Y':2, 'P':1, 'N':0})

#### fence column

In [173]:
df.fence.value_counts()

NA       1651
MnPrv     227
GdPrv      83
GdWo       80
MnWw       10
Name: fence, dtype: int64

In [174]:
df['fence'] = df['fence'].map({'GdPrv':4, 'MnPrv':3, 'GdWo':2, 'MnWw':1, 'NA':0})
df_to_pred['fence'] = df_to_pred['fence'].map({'GdPrv':4, 'MnPrv':3, 'GdWo':2, 'MnWw':1, 'NA':0})

#### overall qual, overall cond,

- Already numerized

#### Lets remove 'utilities' column.

In [175]:
df.drop('utilities', axis = 1, inplace = True)
df_to_pred.drop('utilities', axis = 1, inplace = True)

In [176]:
ordinal_list.remove('utilities')

#### Check data.

In [177]:
df.head()

,ms_subclass,ms_zoning,lot_frontage,lot_area,street,lot_shape,land_contour,lot_config,land_slope,neighborhood,...,enclosed_porch,3ssn_porch,screen_porch,pool_area,fence,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,60,RL,0.0,13517,Pave,2,Lvl,CulDSac,2,Sawyer,...,0,0,0,0,0,0,3,2010,WD,130500
1,60,RL,43.0,11492,Pave,2,Lvl,CulDSac,2,SawyerW,...,0,0,0,0,0,0,4,2009,WD,220000
2,20,RL,68.0,7922,Pave,3,Lvl,Inside,2,NAmes,...,0,0,0,0,0,0,1,2010,WD,109000
3,60,RL,73.0,9802,Pave,3,Lvl,Inside,2,Timber,...,0,0,0,0,0,0,4,2010,WD,174000
4,50,RL,82.0,14235,Pave,2,Lvl,Inside,2,SawyerW,...,0,0,0,0,0,0,3,2010,WD,138500


In [178]:
df_to_pred.head()

,ms_subclass,ms_zoning,lot_frontage,lot_area,street,lot_shape,land_contour,lot_config,land_slope,neighborhood,...,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,fence,misc_val,mo_sold,yr_sold,sale_type
0,190,RM,69.0,9142,Pave,3,Lvl,Inside,2,OldTown,...,60,112,0,0,0,0,0,4,2006,WD
1,90,RL,0.0,9662,Pave,2,Lvl,Inside,2,Sawyer,...,0,0,0,0,0,0,0,8,2006,WD
2,60,RL,58.0,17104,Pave,2,Lvl,Inside,2,Gilbert,...,24,0,0,0,0,0,0,9,2006,New
3,30,RM,60.0,8520,Pave,3,Lvl,Inside,2,OldTown,...,0,184,0,0,0,0,0,7,2007,WD
4,20,RL,0.0,9500,Pave,2,Lvl,Inside,2,NAmes,...,76,0,0,185,0,0,0,7,2009,WD


In [179]:
df.shape

(2051, 75)

In [180]:
df_to_pred.shape

(879, 74)

### Column match.

> - Before we get dummies, lets check if train and test data have same unique values for the columns that we want to get dummies.

In [181]:
# Write a function to check if a column's sorted unique value in train and test data is equal to each other. 
# If they are not equal, print out the column name to check it.
def column_not_ok(df1, df2, column_names):
    for i in column_names:
        if (set(df[i].unique()) != set(df_to_pred[i].unique())) == True:
            print(str(i))    

In [182]:
# Ordinal column names which has different unique value set
column_not_ok(df, df_to_pred, ordinal_list)

overall_qual
bsmt_cond
heating_qc
kitchen_qual
functional
garage_qual


In [183]:
# Nominal column names which has different unique value set
column_not_ok(df, df_to_pred, nominal_list)

ms_subclass
ms_zoning
neighborhood
condition_2
roof_matl
exterior_1st
exterior_2nd
mas_vnr_type
heating
sale_type


In [184]:
# List of column names which has different unique value set.
not_ok = ['overall_qual',
        'bsmt_cond',
        'heating_qc',
        'kitchen_qual',
        'functional',
        'garage_qual',
        'ms_zoning',
        'ms_subclass',
        'neighborhood',
        'condition_2',
        'roof_matl',
        'exterior_1st',
        'exterior_2nd',
        'mas_vnr_type',
        'heating',
        'sale_type']

### Let's compare train and test data of those columns.

- overall_qual
- bsmt_cond
- heating_qc
- kitchen_qual
- functional
- garage_qual
- ms_zoning
- neighborhood
- condition_2
- roof_matl
- exterior_1st
- exterior_2nd
- mas_vnr_type
- heating
- sale_type


> We will try to change some values if their size is small, which won't impact our result.

#### Let's write a function that compare each columns unique values.

In [185]:
def compare_unique(data1, data2, column_name):
    for i in column_name:
        if len(data1[i].unique()) > len(data2[i].unique()):
            print(i)
            print(sorted(data1[i].unique()))
            print(sorted(data2[i].unique()))
            print(data1[i].value_counts().sort_index())
            print('-' * 30)
        else:
            print(i)
            print(sorted(data1[i].unique()))
            print(sorted(data2[i].unique()))
            print(data2[i].value_counts().sort_index())
            print('-' * 30)

In [186]:
compare_unique(df, df_to_pred, not_ok)

overall_qual
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[2, 3, 4, 5, 6, 7, 8, 9, 10]
1       4
2       9
3      29
4     159
5     563
6     506
7     431
8     250
9      77
10     23
Name: overall_qual, dtype: int64
------------------------------
bsmt_cond
[0, 1, 2, 3, 4, 5]
[0, 2, 3, 4]
0      55
1       5
2      65
3    1834
4      89
5       3
Name: bsmt_cond, dtype: int64
------------------------------
heating_qc
[0, 1, 2, 3, 4]
[1, 2, 3, 4]
0       3
1      67
2     597
3     319
4    1065
Name: heating_qc, dtype: int64
------------------------------
kitchen_qual
[1, 2, 3, 4]
[0, 1, 2, 3, 4]
0      1
1     23
2    447
3    354
4     54
Name: kitchen_qual, dtype: int64
------------------------------
functional
[0, 1, 2, 3, 4, 5, 6, 7]
[2, 3, 4, 5, 6, 7]
0       2
1       2
2       7
3      12
4      29
5      42
6      42
7    1915
Name: functional, dtype: int64
------------------------------
garage_qual
[0, 1, 2, 3, 4, 5]
[0, 1, 2, 3, 4]
0     114
1       2
2      82
3    1832
4      18
5 

### Delete columns

#### Write a function that convert the insignificant values in the row to null value. And then drop the null value.

In [187]:
def del_row(data1, data2, column_name):
    for i in column_name:
        if len(set(data1[i].unique()) - set(data2[i].unique())) != 0:
            for j in set(data1[i].unique()) - set(data2[i].unique()):
                data1[i][data1[i]==j] = None
    return data1   

In [188]:
# Applt the function
df = del_row(df, df_to_pred, nominal_list)
df = del_row(df, df_to_pred, ordinal_list)

df_to_pred = del_row(df_to_pred, df, nominal_list)
df_to_pred = del_row(df_to_pred, df, ordinal_list)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [189]:
df.dropna(axis=0, inplace=True)
df.shape

(2003, 75)

#### We still have some columns above. Lets review the data.

In [190]:
nom_ord_list = ordinal_list + nominal_list
column_not_ok(df, df_to_pred, nom_ord_list)

kitchen_qual
garage_cond
roof_matl
exterior_1st
exterior_2nd
mas_vnr_type
heating
sale_type


In [191]:
df['electrical'].value_counts()

4    1833
3     135
2      28
1       7
Name: electrical, dtype: int64

In [192]:
df_to_pred['electrical'].value_counts()

4    814
3     48
2     15
1      2
Name: electrical, dtype: int64

In [193]:
# change 0 value to np.NaN and drop the row.
df_to_pred['electrical'][df_to_pred['electrical'] == 0] = np.NaN

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [194]:
df['garage_cond'].value_counts()

3    1837
0     107
2      41
4      12
1       6
Name: garage_cond, dtype: int64

In [195]:
df_to_pred['garage_cond'].value_counts()

3    797
0     45
2     27
1      6
4      3
5      1
Name: garage_cond, dtype: int64

In [196]:
# change 5 value to np.NaN and drop the row.
df_to_pred['garage_cond'][df_to_pred['garage_cond'] == 5] = 4


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [197]:
df.garage_cond.value_counts()
df_to_pred.garage_cond.value_counts()

3    797
0     45
2     27
1      6
4      4
Name: garage_cond, dtype: int64

### Get dummies by using 'dummies' fuction

In [198]:
# Check if train and test datasets have same dtype to prevent int/floats difference in dummies.
# Lets change all float to integer.
df = df.astype(float, errors='ignore')
df.head()

,ms_subclass,ms_zoning,lot_frontage,lot_area,street,lot_shape,land_contour,lot_config,land_slope,neighborhood,...,enclosed_porch,3ssn_porch,screen_porch,pool_area,fence,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,60.0,RL,0.0,13517.0,Pave,2.0,Lvl,CulDSac,2.0,Sawyer,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2010.0,WD,130500.0
1,60.0,RL,43.0,11492.0,Pave,2.0,Lvl,CulDSac,2.0,SawyerW,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2009.0,WD,220000.0
2,20.0,RL,68.0,7922.0,Pave,3.0,Lvl,Inside,2.0,NAmes,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2010.0,WD,109000.0
3,60.0,RL,73.0,9802.0,Pave,3.0,Lvl,Inside,2.0,Timber,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2010.0,WD,174000.0
4,50.0,RL,82.0,14235.0,Pave,2.0,Lvl,Inside,2.0,SawyerW,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2010.0,WD,138500.0


In [199]:
df_to_pred = df_to_pred.astype(float, errors='ignore')
df_to_pred.head()

,ms_subclass,ms_zoning,lot_frontage,lot_area,street,lot_shape,land_contour,lot_config,land_slope,neighborhood,...,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,fence,misc_val,mo_sold,yr_sold,sale_type
0,190.0,RM,69.0,9142.0,Pave,3.0,Lvl,Inside,2.0,OldTown,...,60.0,112.0,0.0,0.0,0.0,0.0,0.0,4.0,2006.0,WD
1,90.0,RL,0.0,9662.0,Pave,2.0,Lvl,Inside,2.0,Sawyer,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2006.0,WD
2,60.0,RL,58.0,17104.0,Pave,2.0,Lvl,Inside,2.0,Gilbert,...,24.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,2006.0,New
3,30.0,RM,60.0,8520.0,Pave,3.0,Lvl,Inside,2.0,OldTown,...,0.0,184.0,0.0,0.0,0.0,0.0,0.0,7.0,2007.0,WD
4,20.0,RL,0.0,9500.0,Pave,2.0,Lvl,Inside,2.0,NAmes,...,76.0,0.0,0.0,185.0,0.0,0.0,0.0,7.0,2009.0,WD


In [200]:
# Define a function that will return the data with dummies of specific columns.
def dummies(dataframe1, column_name):
    data = pd.get_dummies(dataframe1, columns=column_name)
    return data

In [201]:
df = dummies(df, nom_ord_list)
df_to_pred = dummies(df_to_pred, nom_ord_list)

### Save final cleaned data

In [202]:
df.to_csv("../datasets/train_cleaned_final.csv")
df_to_pred.to_csv('../datasets/test_cleaned_final.csv')